# Upload model to MinIO artifact store

In [2]:
from kfp.components import (
    create_component_from_func,
    InputPath
)
from typing import NamedTuple

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"


def upload_model(
    file_dir: InputPath(str),
    minio_url: str,
    minio_user: str,
    minio_pass: str,
    export_bucket: str = "models",
    model_name: str = "my-model",
    model_version: int = 1,
    model_format: str = "onnx",
) -> NamedTuple(
        "UploadOutput", [
            ("s3_address", str), 
            ("triton_s3_address", str)
        ]):
    """Uploads a model file to MinIO artifact store."""

    import logging
    from collections import namedtuple
    from minio import Minio
    import sys

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format='%(levelname)s %(asctime)s: %(message)s'
    )
    logger = logging.getLogger()

    logger.info(f"Establishing MinIO connection to '{minio_url}'...")
    minio_client = Minio(minio_url,
                         access_key=minio_user,
                         secret_key=minio_pass,
                         secure=False)

    # Create export bucket if it does not yet exist
    response = minio_client.list_buckets()
    export_bucket_exists = False
    for bucket in response:
        if bucket.name == export_bucket:
            export_bucket_exists = True

    if not export_bucket_exists:
        logger.info(f"Creating bucket '{export_bucket}'...")
        minio_client.make_bucket(bucket_name=export_bucket)

    model_path = f"{model_name}/{model_version}/model.{model_format}"
    s3_address = f"s3://{minio_url}/{export_bucket}/{model_format}"
    triton_s3_address = f"{s3_address}/{model_path}"

    logger.info(f"Saving onnx file to MinIO (s3 address: {s3_address})...")
    minio_client.fput_object(
        bucket_name=export_bucket,  # bucket name in Minio
        object_name=f"{model_format}/{model_path}",  # file name in bucket of Minio / for Triton name MUST be model.onnx!
        file_path=file_dir,  # file path / name in local system
    )

    logger.info("Finished.")
    out_tuple = namedtuple("UploadOutput", ["s3_address", "triton_s3_address"])
    return out_tuple(s3_address, triton_s3_address)


upload_model_comp = create_component_from_func(
    func=upload_model,
    output_component_file='component.yaml',
    base_image=BASE_IMAGE
)